In [ ]:
!pip install webdriver_manager
!pip install selenium
!apt-get update
!apt-get -q upgrade -y
!apt-get install chromium-browser
!apt-get -q install -y chromium-chromedriver
!pip install seleniumbase
!pip install easygui
!pip install PyQt5
!pip install tk

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import getpass
import shutil
import easygui
from PyQt5.QtWidgets import QFileDialog, QApplication
import sys
from tkinter import Tk, filedialog

In [ ]:
# Configuração do webdriver
options = webdriver.ChromeOptions()
options.binary_location = '/usr/bin/chromium-browser'
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')

navegador = webdriver.Chrome(options=options)

In [ ]:
def fazer_login():
    # Passo 1 - selecionar o site
    url = "https://www.sei.mg.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0"
    navegador.get(url)

    # Pedir ao usuário para inserir o nome de usuário
    usuario = input("Digite o seu usuário: ")

    # Pedir ao usuário para inserir a senha de forma segura (oculta)
    senha = getpass.getpass("Digite a sua senha: ")

    # Preencher campos de usuário e senha
    navegador.find_element(By.XPATH, '//*[@id="txtEmail"]').send_keys(usuario)
    navegador.find_element(By.XPATH, '//*[@id="pwdSenha"]').send_keys(senha)

    # Passo 3 - clicar no botão e entrar
    navegador.find_element(By.XPATH, '//*[@id="sbmLogin"]').click()

    try:
        # Verificar se o login foi bem-sucedido
        if navegador.find_element(By.ID, 'divInfraBarraLocalizacao').text == 'Controle de Acessos Externos':
            print("Login bem-sucedido!")
        else:
            print("Login falhou. Verifique seu usuário e senha e tente novamente.")
            fazer_login()  # Solicitar que o usuário insira novamente as credenciais
    except NoSuchElementException:
        print("Login falhou. Verifique seu usuário e senha e tente novamente.")
        fazer_login()  # Solicitar que o usuário insira novamente as credenciais

# Iniciar o processo de login
fazer_login()

In [ ]:
# Inicialização de listas para armazenar URLs de processos
urls_processos = []

# Loop para coletar URLs de processos
while True:
    elementos_links_processos = navegador.find_elements(By.XPATH, '//*[@id="tblDocumentos"]//td[2]/a')

    if not elementos_links_processos:
        break

    urls_processos += [elemento.get_attribute('href') for elemento in elementos_links_processos]

    try:
        paginacao = navegador.find_element(By.ID, 'lnkInfraProximaPaginaSuperior')
        paginacao.click()
        time.sleep(3)
    except NoSuchElementException:
        break

# Inicialização de lista para armazenar informações dos processos
informacoes = []

# Loop para coletar informações dos processos
for url_processo in urls_processos:
    try:
        navegador.get(url_processo)

        numero_processo_pagina = navegador.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[2]/td[2]').text
        tabela_cabecalho = navegador.find_element(By.ID, 'tblCabecalho')
        dado1 = tabela_cabecalho.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[3]/td[2]').text
        dado2 = tabela_cabecalho.find_element(By.XPATH, '//*[@id="tblCabecalho"]/tbody/tr[4]/td[2]').text
        tabela_documentos = navegador.find_element(By.ID, 'tblDocumentos')
        elementos_ultima_linha = tabela_documentos.find_elements(By.XPATH, '//*[@id="tblDocumentos"]/tbody/tr[last()]/td')
        dado3 = elementos_ultima_linha[2].text
        dado4 = elementos_ultima_linha[3].text
        dado5 = elementos_ultima_linha[4].text

        informacoes.append({'Número de Processo Página': numero_processo_pagina, 'Tipo de Processo': dado1, 'Data de Criação': dado2, 'Último Documento': dado3, 'Data de Última Movimentação': dado4, 'Unidade do Documento': dado5})
    except Exception as e:
        print(f"Erro ao acessar a URL: {url_processo}")
        print(f"Erro: {e}")
        continue  # Pula para o próximo link em caso de erro

# Criação de um DataFrame com as informações
df = pd.DataFrame(informacoes, columns=['Número de Processo Página', 'Tipo de Processo', 'Data de Criação', 'Último Documento', 'Data de Última Movimentação', 'Unidade do Documento'])

# Nome do arquivo de saída
nome_arquivo = "dados_processos.xlsx"

# Salvar o DataFrame em um arquivo Excel
df.to_excel(nome_arquivo, index=False)


In [ ]:
from google.colab import files

# Mova o arquivo para o diretório /content, caso ainda não esteja lá
import shutil
shutil.move(nome_arquivo, '/content/' + nome_arquivo)

# Gere um link para o usuário baixar o arquivo
files.download(nome_arquivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>